In [3]:
import pickle
import pandas as pd

In [4]:
data_dir = '/lustre/groups/epigenereg01/workspace/projects/vale/mlm/'

In [12]:
folds_df = pd.read_csv(data_dir + 'half_life/agarwal_2022/saluki_paper/Fig3_S4/binnedgenes.txt', sep='\t', usecols=[0,1],
                      names=['Fold','gene_id'], skiprows=1)#folds as they are in Agarwal article

In [6]:
transcript_to_gene = pd.read_csv(data_dir + 'UTR_coords/GRCh38_EnsembleCanonical_HGNC.tsv.gz', sep='\t', 
                                     names=['gene_id','transcript_id'], skiprows=1,usecols=[0,1])

In [7]:
used_transcripts = transcript_to_gene[transcript_to_gene.gene_id.isin(folds_df.gene_id)].transcript_id.values

In [8]:
input_fasta = data_dir + 'fasta/Homo_sapiens_rna.fa'

In [13]:
faidx = pd.read_csv(input_fasta + '.fai', sep='\t',header=None,names=['name','length','offset','linebases','linewidth'])
faidx['transcript_id'] = faidx.name.apply(lambda x:x.split('.')[0])
faidx = faidx[faidx.transcript_id.isin(used_transcripts)].drop(columns='transcript_id')

faidx.name.apply(lambda x:x.split(':')[0]).to_csv(data_dir + 'half_life/agarwal_2022/regions.tsv',index=None,header=None)